# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

This notebook will test production-critical “long-term memory via consolidation” behavior, i.e.:

session history (user/assistant only) → LLM consolidation → UserProfile.memory_entries

UserProfile.memory_entries → LLM generation → UserProfile.system_instructions

persisted profile update → session marked needs_user_instructions_refresh=True

refresh path: SessionManager.get_user_profile_instructions_for_session(session) updates snapshot + clears flag

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# Notebook 09 - Cell 1 (CONFIG)
# Goal: one-cell environment bootstrap, using ONLY classes that exist in INTERGRAX_ENGINE_BUNDLE.py.

import os
from typing import List

from intergrax.llm.messages import ChatMessage
from intergrax.memory.stores.in_memory_conversational_store import InMemoryConversationalMemoryStore
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.memory.user_profile_manager import UserProfileManager
from intergrax.runtime.user_profile.user_profile_instructions_service import UserProfileInstructionsConfig, UserProfileInstructionsService

# Runtime-level consolidation (session -> profile memory entries)
from intergrax.runtime.user_profile.session_memory_consolidation_service import SessionMemoryConsolidationConfig, SessionMemoryConsolidationService

# LLM adapter (real adapter; uses defaults from the adapter itself)
from intergrax.llm_adapters.ollama_adapter import LangChainOllamaAdapter

# ----------------------------
# Identifiers
# ----------------------------
USER_ID = "user_artur"
SESSION_ID = "sess_notebook09_001"

# ----------------------------
# Stores / managers (real framework components)
# ----------------------------
conversational_store = InMemoryConversationalMemoryStore()
user_profile_store = InMemoryUserProfileStore()
user_profile_manager = UserProfileManager(store=user_profile_store)

# ----------------------------
# LLM adapter (real adapter)
# ----------------------------
llm = LangChainOllamaAdapter()

# ----------------------------
# Consolidation service under test
# ----------------------------

instructions_cfg = UserProfileInstructionsConfig(
    regenerate_if_present=False,
)

consolidation_cfg = SessionMemoryConsolidationConfig(
    regenerate_system_instructions=True,
    force_regenerate_system_instructions=False,
)

instructions_service = UserProfileInstructionsService(
    llm=llm,
    manager=user_profile_manager,
    config=instructions_cfg,
)

session_memory_consolidation_service = SessionMemoryConsolidationService(
    llm=llm,
    profile_manager=user_profile_manager,
    instructions_service=instructions_service,
    config=consolidation_cfg,
)

# ----------------------------
# Seed baseline-style history (user/assistant only)
# We'll use this later to trigger consolidation into UserProfile.memory_entries.
# ----------------------------
seed_history: List[ChatMessage] = [
    ChatMessage(role="user", content="I am Artur. I build Intergrax and Mooff. I prefer concise technical answers."),
    ChatMessage(role="assistant", content="Understood. I will keep responses technical and concise."),
    ChatMessage(role="user", content="Never use emojis in code or technical docs. Default project context: Intergrax Drop-In Knowledge Runtime."),
    ChatMessage(role="assistant", content="Acknowledged. No emojis in code/docs; default context set to Intergrax Drop-In Knowledge Runtime."),
]

async def _bootstrap() -> None:
    # Persist seeded conversation into the ConversationalMemoryStore
    memory = await conversational_store.load_memory(session_id=SESSION_ID)
    for m in seed_history:
        await conversational_store.append_message(memory=memory, message=m)

    # Ensure user profile exists
    profile = await user_profile_store.get_profile(USER_ID)

    print("CONFIG READY")
    print(f"- USER_ID   : {USER_ID}")
    print(f"- SESSION_ID: {SESSION_ID}")
    print(f"- seeded_messages: {len(seed_history)}")
    print(f"- profile_exists  : {profile.identity.user_id == USER_ID}")

await _bootstrap()



CONFIG READY
- USER_ID   : user_artur
- SESSION_ID: sess_notebook09_001
- seeded_messages: 4
- profile_exists  : True


# Execute consolidation (history -> LLM -> profile.memory_entries [+ optional system_instructions regeneration])
# and validate production-critical invariants.

In [2]:
async def _run_consolidation_and_validate() -> None:
    # Load conversation memory (history source for consolidation)
    memory_before = await conversational_store.load_memory(session_id=SESSION_ID)
    messages_before = list(memory_before.get_all())

    # Basic history invariant
    roles_before = [m.role for m in messages_before]
    assert all(r in ("user", "assistant") for r in roles_before), (
        "History invariant failed: only user/assistant roles should be present."
    )
    assert len(messages_before) > 0, "Expected non-empty conversation history."

    # Load profile before
    profile_before = await user_profile_store.get_profile(USER_ID)
    entries_before = list(profile_before.memory_entries)
    instr_before = profile_before.system_instructions

    # Run consolidation (BUNDLE signature: user_id, session_id, messages)
    created_entries = await session_memory_consolidation_service.consolidate_session(
        user_id=USER_ID,
        session_id=SESSION_ID,
        messages=messages_before,
    )

    # Reload profile after (persistence check)
    profile_after = await user_profile_store.get_profile(USER_ID)
    entries_after = list(profile_after.memory_entries)
    instr_after = profile_after.system_instructions

    # Assertions: consolidation returned entries
    assert isinstance(created_entries, list), "Expected list result from consolidate_session."
    assert len(created_entries) > 0, "Expected at least one created memory entry from consolidation."

    # Assertions: persisted entries exist (and did not shrink)
    assert len(entries_after) >= len(entries_before), (
        "Expected memory entries count to not decrease after consolidation."
    )
    assert len(entries_after) > 0, "Expected at least one memory entry persisted in the profile."

    # If regeneration is enabled, instructions should be set (non-empty).
    assert isinstance(instr_after, str) and instr_after.strip(), (
        "Expected non-empty UserProfile.system_instructions after consolidation (regeneration enabled)."
    )

    # --- NEW: Production contract for stable entry IDs ---
    # 1) created entries must have entry_id (str, non-empty)
    created_ids = []
    for e in created_entries:
        assert hasattr(e, "entry_id"), "Memory entry missing entry_id attribute."
        assert isinstance(e.entry_id, str), f"Expected entry_id to be str, got {type(e.entry_id).__name__}"
        assert e.entry_id.strip(), "Expected non-empty entry_id."
        created_ids.append(e.entry_id)

    assert len(set(created_ids)) == len(created_ids), "Expected unique entry_id values among created entries."

    # 2) persisted entries must have entry_id (str, non-empty, unique)
    persisted_ids = []
    for e in entries_after:
        assert hasattr(e, "entry_id"), "Persisted memory entry missing entry_id attribute."
        assert isinstance(e.entry_id, str), f"Expected persisted entry_id to be str, got {type(e.entry_id).__name__}"
        assert e.entry_id.strip(), "Expected non-empty persisted entry_id."
        persisted_ids.append(e.entry_id)

    assert len(set(persisted_ids)) == len(persisted_ids), "Expected unique entry_id values among persisted entries."

    # 3) created IDs should exist in persisted IDs (strong persistence signal)
    missing = [x for x in created_ids if x not in set(persisted_ids)]
    assert not missing, f"Some created entry_ids were not persisted: {missing[:3]}"

    # History must remain unchanged (consolidation must not mutate persisted conversation history)
    memory_after = await conversational_store.load_memory(session_id=SESSION_ID)
    messages_after = list(memory_after.get_all())

    roles_after = [m.role for m in messages_after]
    assert all(r in ("user", "assistant") for r in roles_after), (
        "History invariant failed after consolidation: system messages must not be persisted."
    )
    assert len(messages_after) == len(messages_before), (
        "Expected consolidation to not mutate persisted conversation history."
    )

    # Output summary
    print("CONSOLIDATION OK")
    print(f"- messages_count: {len(messages_before)}")
    print(f"- memory_entries_before: {len(entries_before)}")
    print(f"- memory_entries_after : {len(entries_after)}")
    print(f"- created_entries      : {len(created_entries)}")
    print(f"- created_entry_ids_ok : {len(created_ids)}")
    print(f"- persisted_entry_ids_ok: {len(persisted_ids)}")
    print(f"- instructions_before_empty: {not bool((instr_before or '').strip())}")
    print(f"- instructions_after_len   : {len(instr_after)}")

await _run_consolidation_and_validate()


CONSOLIDATION OK
- messages_count: 4
- memory_entries_before: 0
- memory_entries_after : 4
- created_entries      : 4
- created_entry_ids_ok : 4
- persisted_entry_ids_ok: 4
- instructions_before_empty: True
- instructions_after_len   : 689


# Re-run consolidation on the same session history and validate:
# - history unchanged
# - memory growth behavior is observable (idempotency / no-spam signal)

In [3]:
async def _rerun_consolidation_and_measure_growth() -> None:
    # Load history once (must not change)
    memory = await conversational_store.load_memory(session_id=SESSION_ID)
    messages = list(memory.get_all())
    roles = [m.role for m in messages]
    assert all(r in ("user", "assistant") for r in roles), "History roles invariant failed."

    # Profile snapshot before second run
    profile_before = await user_profile_store.get_profile(USER_ID)
    before_count = len(profile_before.memory_entries)
    before_instr = profile_before.system_instructions

    # Second consolidation run
    created_entries_2 = await session_memory_consolidation_service.consolidate_session(
        user_id=USER_ID,
        session_id=SESSION_ID,
        messages=messages,
    )

    # Profile snapshot after second run
    profile_after = await user_profile_store.get_profile(USER_ID)
    after_count = len(profile_after.memory_entries)
    after_instr = profile_after.system_instructions

    # History still unchanged
    memory_after = await conversational_store.load_memory(session_id=SESSION_ID)
    messages_after = list(memory_after.get_all())
    assert len(messages_after) == len(messages), "History mutated by consolidation (must not happen)."

    # Growth accounting
    created_2 = len(created_entries_2)
    delta = after_count - before_count

    # In a perfect world: created_2 == 0 and delta == 0.
    # For now we just measure and ensure consistency (no hidden mutation).
    assert delta >= 0, "Memory entries count should not decrease after consolidation."
    assert created_2 >= 0, "Created entries count must be non-negative."

    print("RE-CONSOLIDATION MEASURED")
    print(f"- memory_entries_before: {before_count}")
    print(f"- memory_entries_after : {after_count}")
    print(f"- delta                : {delta}")
    print(f"- created_entries_2    : {created_2}")
    print(f"- instructions_len_before: {len(before_instr or '')}")
    print(f"- instructions_len_after : {len(after_instr or '')}")

await _rerun_consolidation_and_measure_growth()

RE-CONSOLIDATION MEASURED
- memory_entries_before: 4
- memory_entries_after : 9
- delta                : 5
- created_entries_2    : 5
- instructions_len_before: 689
- instructions_len_after : 689


# Test mid-session consolidation gating in SessionManager:
# - consolidation triggers only on exact multiples of interval
# - cooldown blocks repeated consolidations even if interval matches

In [4]:
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import (
    InMemorySessionStorage,
)
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

# We'll use a fresh user+session to avoid mixing with previous notebook runs.
USER_ID_2 = "user_artur_notebook09_gating"
SESSION_ID_2 = "sess_notebook09_002"

async def _test_mid_session_gating() -> None:
    # Ensure profile exists (in the same in-memory store used by the notebook)
    _ = await user_profile_store.get_profile(USER_ID_2)

    storage = InMemorySessionStorage()

    # interval=2 means consolidation attempt at user_turns: 2, 4, 6...
    # cooldown is set very high so only the first eligible consolidation should run.
    mgr = SessionManager(
        storage,
        user_profile_manager=user_profile_manager,
        session_memory_consolidation_service=session_memory_consolidation_service,
        user_turns_consolidation_interval=2,
        consolidation_cooldown_seconds=999999,
    )

    # Create session
    session = await mgr.create_session(session_id=SESSION_ID_2, user_id=USER_ID_2)

    # Baseline counts
    prof_before = await user_profile_store.get_profile(USER_ID_2)
    entries_before = len(prof_before.memory_entries)

    # Append messages: 4 user turns (with assistant replies), so eligible turns: 2 and 4.
    # Due to cooldown, only turn=2 should consolidate.
    await mgr.append_message(SESSION_ID_2, ChatMessage(role="user", content="U1: My name is Artur."))
    await mgr.append_message(SESSION_ID_2, ChatMessage(role="assistant", content="A1: Noted."))

    await mgr.append_message(SESSION_ID_2, ChatMessage(role="user", content="U2: I prefer concise technical answers."))
    await mgr.append_message(SESSION_ID_2, ChatMessage(role="assistant", content="A2: Understood."))

    await mgr.append_message(SESSION_ID_2, ChatMessage(role="user", content="U3: Never use emojis in code/docs."))
    await mgr.append_message(SESSION_ID_2, ChatMessage(role="assistant", content="A3: Acknowledged."))

    await mgr.append_message(SESSION_ID_2, ChatMessage(role="user", content="U4: Default context is Intergrax Drop-In Knowledge Runtime."))
    await mgr.append_message(SESSION_ID_2, ChatMessage(role="assistant", content="A4: Confirmed."))

    # Reload session and profile
    session_after = await mgr.get_session(SESSION_ID_2)
    prof_after = await user_profile_store.get_profile(USER_ID_2)
    entries_after = len(prof_after.memory_entries)

    # Assertions: user_turns counted correctly
    assert session_after.user_turns == 4, f"Expected 4 user turns, got {session_after.user_turns}"

    # Assertions: consolidated exactly once at turn=2 (cooldown blocks turn=4)
    assert session_after.last_consolidated_turn == 2, (
        f"Expected last_consolidated_turn=2 due to cooldown, got {session_after.last_consolidated_turn}"
    )
    assert session_after.last_consolidated_reason is not None, (
        "Expected last_consolidated_reason to be set after consolidation."
    )
    assert session_after.needs_user_instructions_refresh is True, (
        "Expected needs_user_instructions_refresh=True after consolidation."
    )

    # Memory growth happened at least once
    assert entries_after > entries_before, (
        "Expected profile.memory_entries to increase after mid-session consolidation."
    )

    print("MID-SESSION GATING OK")
    print(f"- session.user_turns          : {session_after.user_turns}")
    print(f"- last_consolidated_turn      : {session_after.last_consolidated_turn}")
    print(f"- last_consolidated_reason    : {session_after.last_consolidated_reason}")
    print(f"- needs_user_instructions_refresh: {session_after.needs_user_instructions_refresh}")
    print(f"- memory_entries_before       : {entries_before}")
    print(f"- memory_entries_after        : {entries_after}")
    print(f"- delta                       : {entries_after - entries_before}")

await _test_mid_session_gating()

MID-SESSION GATING OK
- session.user_turns          : 4
- last_consolidated_turn      : 2
- last_consolidated_reason    : mid_session
- needs_user_instructions_refresh: True
- memory_entries_before       : 0
- memory_entries_after        : 1
- delta                       : 1


# Validate snapshot refresh path after mid-session consolidation.

In [5]:
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import (
    InMemorySessionStorage,
)
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

USER_ID_3 = "user_artur_notebook09_mid_refresh"
SESSION_ID_3 = "sess_notebook09_003"

async def _mid_session_gating_then_refresh() -> None:
    # Ensure profile exists
    _ = await user_profile_store.get_profile(USER_ID_3)

    # IMPORTANT: single storage instance for the whole flow
    storage = InMemorySessionStorage()

    mgr = SessionManager(
        storage,
        user_profile_manager=user_profile_manager,
        session_memory_consolidation_service=session_memory_consolidation_service,
        user_turns_consolidation_interval=2,
        consolidation_cooldown_seconds=999999,
    )

    # Create session
    _ = await mgr.create_session(session_id=SESSION_ID_3, user_id=USER_ID_3)

    # Drive user_turns to 2 (eligible for consolidation)
    await mgr.append_message(SESSION_ID_3, ChatMessage(role="user", content="U1: I am Artur."))
    await mgr.append_message(SESSION_ID_3, ChatMessage(role="assistant", content="A1: Noted."))

    await mgr.append_message(SESSION_ID_3, ChatMessage(role="user", content="U2: Never use emojis in code/docs."))
    await mgr.append_message(SESSION_ID_3, ChatMessage(role="assistant", content="A2: Understood."))

    session_before = await mgr.get_session(SESSION_ID_3)
    assert session_before is not None, "Session not found."
    assert session_before.user_turns == 2, f"Expected user_turns=2, got {session_before.user_turns}"
    assert session_before.last_consolidated_turn == 2, (
        f"Expected consolidation at turn 2, got {session_before.last_consolidated_turn}"
    )
    assert session_before.needs_user_instructions_refresh is True, (
        "Expected needs_user_instructions_refresh=True after consolidation."
    )

    # Refresh snapshot (official API)
    refreshed_text = await mgr.get_user_profile_instructions_for_session(session_before)
    assert isinstance(refreshed_text, str) and refreshed_text.strip(), "Expected non-empty refreshed instructions."

    session_after = await mgr.get_session(SESSION_ID_3)
    assert session_after is not None, "Session not found after refresh."
    profile_after = await user_profile_store.get_profile(USER_ID_3)

    assert session_after.needs_user_instructions_refresh is False, (
        "Expected refresh flag cleared after snapshot refresh."
    )
    assert session_after.user_profile_instructions == profile_after.system_instructions, (
        "Expected session snapshot to match persisted profile.system_instructions."
    )

    print("MID-SESSION REFRESH OK")
    print(f"- user_turns                 : {session_after.user_turns}")
    print(f"- last_consolidated_turn     : {session_after.last_consolidated_turn}")
    print(f"- needs_user_instructions_refresh: {session_after.needs_user_instructions_refresh}")
    print(f"- snapshot_len               : {len(session_after.user_profile_instructions or '')}")

await _mid_session_gating_then_refresh()


MID-SESSION REFRESH OK
- user_turns                 : 2
- last_consolidated_turn     : 2
- needs_user_instructions_refresh: False
- snapshot_len               : 634


# Validate: deleted memory entries are ignored in instruction regeneration
# and can be physically removed via purge_deleted_memory_entries.

In [7]:
async def _soft_delete_and_purge_contract() -> None:
    user_id = "user_artur_notebook09_soft_delete"
    session_id = "sess_notebook09_004"

    # Ensure profile exists
    _ = await user_profile_store.get_profile(user_id)

    # Seed a small conversation and consolidate once
    mem = await conversational_store.load_memory(session_id=session_id)
    await conversational_store.append_message(mem, ChatMessage(role="user", content="I am Artur."))
    await conversational_store.append_message(mem, ChatMessage(role="assistant", content="Noted."))
    await conversational_store.append_message(mem, ChatMessage(role="user", content="Never use emojis in code/docs."))
    await conversational_store.append_message(mem, ChatMessage(role="assistant", content="Understood."))

    messages = list((await conversational_store.load_memory(session_id=session_id)).get_all())

    created_1 = await session_memory_consolidation_service.consolidate_session(
        user_id=user_id,
        session_id=session_id,
        messages=messages,
    )
    assert isinstance(created_1, list) and len(created_1) > 0, (
        "Expected consolidation to return a non-empty list of memory entries."
    )

    # Validate entry_id contract on returned entries (production requirement)
    for e in created_1:
        assert isinstance(e.entry_id, str) and e.entry_id.strip(), "Expected non-empty entry_id (str)."

    profile1 = await user_profile_store.get_profile(user_id)
    assert len(profile1.memory_entries) > 0, "Expected persisted entries after consolidation."
    assert isinstance(profile1.system_instructions, str) and profile1.system_instructions.strip(), (
        "Expected system_instructions after consolidation."
    )

    instr_before = profile1.system_instructions
    entries_before_delete = len(profile1.memory_entries)

    # Pick an active entry WITH entry_id
    target = next(
        (e for e in profile1.memory_entries if (not e.deleted) and isinstance(e.entry_id, str) and e.entry_id.strip()),
        None,
    )
    assert target is not None, "No active entry with entry_id found to delete."

    target_id = target.entry_id
    target_content = target.content or ""

    # Soft-delete via official manager API (no direct mutations)
    await user_profile_manager.remove_memory_entry(user_id=user_id, entry_id=target_id)

    profile2 = await user_profile_store.get_profile(user_id)
    deleted_entry = next((e for e in profile2.memory_entries if e.entry_id == target_id), None)
    assert deleted_entry is not None, "Deleted entry not found after remove_memory_entry."
    assert deleted_entry.deleted is True, "Expected deleted=True after remove_memory_entry."

    # Trigger instruction regeneration via consolidation (second run)
    created_2 = await session_memory_consolidation_service.consolidate_session(
        user_id=user_id,
        session_id=session_id,
        messages=messages,
    )
    assert isinstance(created_2, list), "Expected list result from consolidate_session (second run)."

    profile3 = await user_profile_store.get_profile(user_id)
    instr_after = profile3.system_instructions
    assert isinstance(instr_after, str) and instr_after.strip(), "Expected non-empty instructions after second run."

    # Best-effort leak check: deleted entry content should not appear in regenerated instructions
    # (If content is too short/common, this check may be noisy; we guard with a minimum length.)
    if target_content and len(target_content) >= 12:
        assert target_content not in instr_after, (
            "Soft delete contract failed: deleted entry content leaked into system instructions."
        )

    # Purge maintenance (hard cleanup)
    await user_profile_manager.purge_deleted_memory_entries(user_id=user_id)

    profile4 = await user_profile_store.get_profile(user_id)
    ids_after_purge = {e.entry_id for e in profile4.memory_entries if isinstance(e.entry_id, str)}

    assert target_id not in ids_after_purge, "Expected deleted entry to be physically removed after purge."

    print("SOFT DELETE + PURGE OK (entry_id)")
    print(f"- entries_after_first_consolidation : {entries_before_delete}")
    print(f"- deleted_entry_id                  : {target_id}")
    print(f"- created_entries_second_run        : {len(created_2)}")
    print(f"- instructions_len_before           : {len(instr_before)}")
    print(f"- instructions_len_after            : {len(instr_after)}")
    print(f"- entries_after_purge               : {len(profile4.memory_entries)}")

await _soft_delete_and_purge_contract()


SOFT DELETE + PURGE OK (entry_id)
- entries_after_first_consolidation : 9
- deleted_entry_id                  : 50cb4a1926284f019d8e596d09b8ac1a
- created_entries_second_run        : 2
- instructions_len_before           : 1123
- instructions_len_after            : 1123
- entries_after_purge               : 10


# Multi-session simulation (2–3 sessions) with:
# - session history (user/assistant only) persisted per session
# - explicit consolidation after each session (except the last)
# - regenerated UserProfile.system_instructions used in the next session
# - detailed logs: session messages, profile memory entries, consolidation outputs

In [8]:
from typing import List, Sequence
from intergrax.llm.messages import ChatMessage

from intergrax.memory.stores.in_memory_conversational_store import InMemoryConversationalMemoryStore
from intergrax.memory.stores.in_memory_user_profile_store import InMemoryUserProfileStore
from intergrax.memory.user_profile_manager import UserProfileManager

from intergrax.runtime.user_profile.user_profile_instructions_service import (
    UserProfileInstructionsConfig,
    UserProfileInstructionsService,
)
from intergrax.runtime.user_profile.session_memory_consolidation_service import (
    SessionMemoryConsolidationConfig,
    SessionMemoryConsolidationService,
)

from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

from intergrax.llm_adapters.ollama_adapter import LangChainOllamaAdapter


# ----------------------------
# CONFIG: sessions as List[List[ChatMessage]]
# ----------------------------
USER_ID = "user_artur"
SESSIONS: List[List[ChatMessage]] = [
    [
        ChatMessage(role="user", content="I am Artur. I build Intergrax and Mooff."),
        ChatMessage(role="assistant", content="Understood."),
        ChatMessage(role="user", content="I prefer concise, technical answers. Never use emojis in code/docs."),
        ChatMessage(role="assistant", content="Acknowledged."),
    ],
    [
        ChatMessage(role="user", content="Let's continue. Remember: default project context is Intergrax Drop-In Knowledge Runtime."),
        ChatMessage(role="assistant", content="Got it."),
        ChatMessage(role="user", content="I want deterministic, production-grade outputs."),
        ChatMessage(role="assistant", content="Understood."),
    ],
    # Uncomment to simulate 3rd session
    # [
    #     ChatMessage(role="user", content="Now test if you remember my preferences without repeating them back."),
    #     ChatMessage(role="assistant", content="OK."),
    # ],
]


# ----------------------------
# Framework components (real, no fakes)
# ----------------------------
conversational_store = InMemoryConversationalMemoryStore()
user_profile_store = InMemoryUserProfileStore()
user_profile_manager = UserProfileManager(store=user_profile_store)

llm = LangChainOllamaAdapter()

instructions_cfg = UserProfileInstructionsConfig(
    regenerate_if_present=False,
)

consolidation_cfg = SessionMemoryConsolidationConfig(
    regenerate_system_instructions=True,
    force_regenerate_system_instructions=False,
)

instructions_service = UserProfileInstructionsService(
    llm=llm,
    manager=user_profile_manager,
    config=instructions_cfg,
)

session_memory_consolidation_service = SessionMemoryConsolidationService(
    llm=llm,
    profile_manager=user_profile_manager,
    instructions_service=instructions_service,
    config=consolidation_cfg,
)

# Session manager for snapshot + refresh semantics (runtime-like behavior)
session_storage = InMemorySessionStorage()
session_manager = SessionManager(
    session_storage,
    user_profile_manager=user_profile_manager,
    session_memory_consolidation_service=session_memory_consolidation_service,
    user_turns_consolidation_interval=999999,  # disable auto mid-session consolidation for this simulation
    consolidation_cooldown_seconds=999999,
)


# ----------------------------
# Helpers: pretty logging
# ----------------------------
def _print_header(title: str) -> None:
    print("")
    print("=" * 88)
    print(title)
    print("=" * 88)

def _print_messages(messages: Sequence[ChatMessage]) -> None:
    for i, m in enumerate(messages, start=1):
        content = (m.content or "").strip()
        print(f"  [{i:02d}] {m.role}: {content}")

def _print_memory_entries(profile) -> None:
    entries = list(profile.memory_entries)
    print(f"- memory_entries_count: {len(entries)}")
    if not entries:
        return
    for i, e in enumerate(entries, start=1):
        eid = getattr(e, "entry_id", None)
        deleted = getattr(e, "deleted", False)
        content = (getattr(e, "content", "") or "").strip()
        if len(content) > 140:
            content = content[:140] + "..."
        print(f"  [{i:02d}] id={eid} deleted={deleted} content='{content}'")

def _system_preview(session_snapshot: str) -> str:
    """
    Runtime SYSTEM message preview for the session.
    Note: This is NOT persisted in history.
    """
    text = (session_snapshot or "").strip()
    if not text:
        return "<EMPTY>"
    return text


# ----------------------------
# Core algorithm: iterate sessions, consolidate after each (except last)
# ----------------------------
async def _simulate_sessions() -> None:
    # Ensure profile exists at start
    profile0 = await user_profile_store.get_profile(USER_ID)

    _print_header("SIMULATION START")
    print(f"- USER_ID: {USER_ID}")
    print(f"- sessions_count: {len(SESSIONS)}")
    print(f"- initial_profile_instructions_len: {len((profile0.system_instructions or '').strip())}")
    print(f"- initial_profile_memory_entries  : {len(profile0.memory_entries)}")

    for idx, session_messages in enumerate(SESSIONS, start=1):
        session_id = f"sess_sim_{idx:03d}"

        _print_header(f"SESSION {idx} START  | session_id={session_id}")

        # Create a runtime session and load the snapshot instructions (runtime-like behavior)
        session = await session_manager.create_session(session_id=session_id, user_id=USER_ID)

        # This call is the "official" way to hydrate the snapshot in a deterministic way
        snapshot_text = await session_manager.get_user_profile_instructions_for_session(session)

        profile_before = await user_profile_store.get_profile(USER_ID)
        print("A) PROFILE BEFORE SESSION")
        print(f"- profile.system_instructions_len: {len((profile_before.system_instructions or '').strip())}")
        print(f"- session.user_profile_instructions_len (snapshot): {len((snapshot_text or '').strip())}")
        print("- SYSTEM message preview (runtime-only):")
        print(_system_preview(snapshot_text))

        print("")
        print("B) SESSION HISTORY (to be persisted, user/assistant only)")
        _print_messages(session_messages)

        # Persist session messages into conversational store (history source for consolidation)
        mem = await conversational_store.load_memory(session_id=session_id)
        for m in session_messages:
            # Strict invariant: only user/assistant for persisted history
            assert m.role in ("user", "assistant"), "Only user/assistant roles may be persisted."
            await conversational_store.append_message(mem, m)

        # Reload persisted history and show it
        mem_after_write = await conversational_store.load_memory(session_id=session_id)
        persisted = list(mem_after_write.get_all())

        print("")
        print("C) PERSISTED HISTORY CHECK (must be unchanged roles)")
        roles = [m.role for m in persisted]
        assert all(r in ("user", "assistant") for r in roles), "Persisted history role invariant failed."
        print(f"- persisted_messages_count: {len(persisted)}")
        _print_messages(persisted)

        # Consolidate after this session, except for the last one
        should_consolidate = idx < len(SESSIONS)
        if should_consolidate:
            _print_header(f"SESSION {idx} CONSOLIDATION  | session_id={session_id}")

            profile_pre_consol = await user_profile_store.get_profile(USER_ID)
            print("D1) PROFILE BEFORE CONSOLIDATION")
            print(f"- system_instructions_len: {len((profile_pre_consol.system_instructions or '').strip())}")
            _print_memory_entries(profile_pre_consol)

            created = await session_memory_consolidation_service.consolidate_session(
                user_id=USER_ID,
                session_id=session_id,
                messages=persisted,
            )

            assert isinstance(created, list), "Expected consolidate_session to return a list."
            print("")
            print("D2) CONSOLIDATION RESULT (created entries)")
            print(f"- created_entries_count: {len(created)}")
            for i, e in enumerate(created, start=1):
                eid = getattr(e, "entry_id", None)
                deleted = getattr(e, "deleted", False)
                content = (getattr(e, "content", "") or "").strip()
                if len(content) > 140:
                    content = content[:140] + "..."
                print(f"  [{i:02d}] id={eid} deleted={deleted} content='{content}'")

            profile_post_consol = await user_profile_store.get_profile(USER_ID)
            print("")
            print("D3) PROFILE AFTER CONSOLIDATION (persisted long-term memory + instructions)")
            print(f"- system_instructions_len: {len((profile_post_consol.system_instructions or '').strip())}")
            _print_memory_entries(profile_post_consol)

            # Production note: The next session should see this via snapshot hydration.
            print("")
            print("D4) NOTE")
            print("- The next session should start with this updated profile.system_instructions (via snapshot).")
        else:
            _print_header(f"SESSION {idx} END (no consolidation, last session)")
            print("- No consolidation executed for the last session by design.")

    _print_header("SIMULATION END")
    profile_end = await user_profile_store.get_profile(USER_ID)
    print(f"- final_system_instructions_len: {len((profile_end.system_instructions or '').strip())}")
    print(f"- final_memory_entries_count   : {len(profile_end.memory_entries)}")


await _simulate_sessions()


SIMULATION START
- USER_ID: user_artur
- sessions_count: 2
- initial_profile_instructions_len: 0
- initial_profile_memory_entries  : 0

SESSION 1 START  | session_id=sess_sim_001
A) PROFILE BEFORE SESSION
- profile.system_instructions_len: 0
- session.user_profile_instructions_len (snapshot): 47
- SYSTEM message preview (runtime-only):
You are talking to a user with id 'user_artur'.

B) SESSION HISTORY (to be persisted, user/assistant only)
  [01] user: I am Artur. I build Intergrax and Mooff.
  [02] assistant: Understood.
  [03] user: I prefer concise, technical answers. Never use emojis in code/docs.
  [04] assistant: Acknowledged.

C) PERSISTED HISTORY CHECK (must be unchanged roles)
- persisted_messages_count: 4
  [01] user: I am Artur. I build Intergrax and Mooff.
  [02] assistant: Understood.
  [03] user: I prefer concise, technical answers. Never use emojis in code/docs.
  [04] assistant: Acknowledged.

SESSION 1 CONSOLIDATION  | session_id=sess_sim_001
D1) PROFILE BEFORE CONSO